In [22]:
import numpy as np
import pandas as pd

from claspy.data_loader import load_tssb_dataset, tssb_dataset_names
from claspy.segmentation import BinaryClaSPSegmentation
from claspy.tests.evaluation import covering

In [3]:
tssb = load_tssb_dataset()
tssb

,dataset,window_size,cps,time_series
0,Adiac,10,"[572, 1012, 1232]","[1.681146, 1.383394, 1.029389, 0.688386, 0.354..."
1,ArrowHead,10,[753],"[-1.957721, -1.858082, -1.657983, -1.420794, -..."
2,Beef,50,[705],"[-0.380818, -0.352238, -0.295392, -0.225041, -..."
3,BeetleFly,10,[1280],"[1.114805, 0.774403, 0.477129, 0.115781, -0.24..."
4,BirdChicken,20,[1280],"[1.697714, 1.439032, 1.22287, 0.99708, 0.76479..."
...,...,...,...,...
70,UWaveGestureLibraryX,10,"[600, 1131, 1652, 2193]","[-0.889766, 0.722005, 0.869297, -0.553167, -0...."
71,UWaveGestureLibraryY,10,"[600, 1131, 1652, 2193]","[-1.523467, -0.106794, 0.190361, 0.979247, 0.3..."
72,UWaveGestureLibraryZ,10,"[600, 1131, 1652, 2193]","[-0.527058, 0.416111, -0.524027, 0.41141, 0.12..."
73,WordSynonyms,10,"[202, 2227]","[-1.183581, -0.514028, -0.480539, -0.394855, -..."


Function that calculates the covering scores for each time series in the ```tssb``` using ClaSP with the specified distance measure. Options are
- znormed_euclidean_distance
- cinvariant_euclidean_distance
- euclidean_distance

In [21]:
def covering_scores_for_dist(dist: str) -> list[float]:
    cov_scores = []
    for idx, (dataset, window_size, cps, time_series) in list(tssb.iterrows()):
        clasp = BinaryClaSPSegmentation(distance=dist)
        found_cps = clasp.fit_predict(time_series)
        set_score = float(np.round(covering({0: cps}, found_cps, time_series.shape[0]), 5))
        cov_scores.append(set_score)
    return cov_scores

In [27]:
dist_measures = ["znormed_euclidean_distance", "cinvariant_euclidean_distance", "euclidean_distance"]
df = pd.DataFrame({"Dataset": tssb_dataset_names})

for dm in dist_measures:
    scores = covering_scores_for_dist(dm)
    df[dm] = scores

df

/home/jasper/bachelor/oracle/oracle-venv/lib/python3.10/site-packages/claspy/validation.py:74: RuntimeWarning: invalid value encountered in scalar divide
  z = (s - expected) / np.sqrt(n1 * n2 * (n1 + n2 + 1) / 12.0)
/home/jasper/bachelor/oracle/oracle-venv/lib/python3.10/site-packages/claspy/validation.py:74: RuntimeWarning: invalid value encountered in scalar divide
  z = (s - expected) / np.sqrt(n1 * n2 * (n1 + n2 + 1) / 12.0)


,Dataset,znormed_euclidean_distance,cinvariant_euclidean_distance,euclidean_distance
0,Adiac,0.80744,0.93578,0.94104
1,ArrowHead,0.98550,0.98681,0.98681
2,Beef,0.97340,0.80260,0.80260
3,BeetleFly,0.97530,0.66069,0.65420
4,BirdChicken,0.99377,0.54570,0.54453
...,...,...,...,...
70,UWaveGestureLibraryX,0.98663,0.98734,0.90162
71,UWaveGestureLibraryY,0.96282,0.96206,0.95946
72,UWaveGestureLibraryZ,0.81048,0.79963,0.80961
73,WordSynonyms,0.72692,0.83742,0.72692


In [48]:
znorm = float(np.round(df["znormed_euclidean_distance"].mean(), 5))
cinv  = float(np.round(df["cinvariant_euclidean_distance"].mean(), 5))
euc   = float(np.round(df["euclidean_distance"].mean(), 5))

znorm, cinv, euc

(0.85456, 0.858, 0.84062)